In [1]:
"""
Purpose: Will add any synapse labels with error to synapse exclude

"""

'\nPurpose: Will add any synapse labels with error to synapse exclude\n\n'

In [2]:
import datajoint as dj
import numpy as np

dj.config["display.limit"] = 20

In [3]:
schema = dj.schema("microns_pinky")

Connecting celiib@10.28.0.34:3306


In [4]:
pinky = dj.create_virtual_module("pinky","microns_pinky")

In [5]:
"""Old way for doing it in ta3p100"""
# #things need to include in new Synapse Exclude
# """
# 1) Those with ErrorLabel of SynapseCompartmentLabelFinal
# 2) The old SynapseExclude synapse which aren't form the earlier SynapseCompartLabel error labels migration

# """

# #1) get the synapses with Error label for both the original proofreading session and the new proofreading session
# synapse_ids_error_proof_original = (ta3p100.SynapseCompartmentLabel() & "segmentation=2" & "postsynaptic_label=10").fetch("synapse_id")
# print(len(synapse_ids_error_proof_original))
# synapse_ids_error_proof_new = (ta3p100.SynapseCompartmentLabelFinal() & "segmentation=2" & "postsynaptic_label=10").fetch("synapse_id")
# print(len(synapse_ids_error_proof_new))

'Old way for doing it in ta3p100'

In [10]:
"""Old way for doing it in ta3p100"""
#things need to include in new Synapse Exclude
"""
1) Those with ErrorLabel of SynapseCompartmentLabelFinal
2) The old SynapseExclude synapse which aren't form the earlier SynapseCompartLabel error labels migration

"""

#1) get the synapses with Error label for both the original proofreading session and the new proofreading session
synapse_ids_error_proof_compartment = (pinky.SynapseCompartmentLabelFinalThresholdRevised() & "segmentation=3" & "postsynaptic_label=10").fetch("synapse_id")
print(len(synapse_ids_error_proof_compartment))
synapse_ids_error_proof_spine = (pinky.SynapseSpineLabelFinalThresholdRevised() & "segmentation=3" & "spine_label=10").fetch("synapse_id")
print(len(synapse_ids_error_proof_spine))



13259
0


In [11]:
# pinky.ExclusionCriteria.insert1(dict(criteria_id=1,criteria_desc="postsyn label was error in labeling pipeline"))

In [12]:
pinky.SynapseExclude.describe()

# Excluded anatomically localized synapse between two Segments
-> pinky.Synapse
-> pinky.ExclusionCriteria



'# Excluded anatomically localized synapse between two Segments\n-> pinky.Synapse\n-> pinky.ExclusionCriteria\n'

In [14]:
#inserting all the error from the proofreading stage 2 into new synapseExclude table
from tqdm import tqdm

total_dict = []
synapse_errors_compartment = [dict(segmentation=3,synapse_id=sg,criteria_id=1) for sg in synapse_ids_error_proof_compartment]
len(synapse_errors_compartment)
if len(synapse_errors_compartment)>0:
    print(synapse_errors_compartment[0])
#insert all into segment exclude
pinky.SynapseExclude.insert(synapse_errors_compartment,skip_duplicates=True)

synapse_errors_spine = [dict(segmentation=3,synapse_id=sg,criteria_id=1) for sg in synapse_ids_error_proof_spine]
len(synapse_errors_spine)
if len(synapse_errors_spine)>0:
    print(synapse_errors_spine[0])
#insert all into segment exclude
pinky.SynapseExclude.insert(synapse_errors_spine,skip_duplicates=True)




{'segmentation': 3, 'synapse_id': 60, 'criteria_id': 1}


In [ ]:
# import collections

# final_added_synapses = []

# for q in tqdm(original_synapse_exclude):
#     if q["synapse_id"] not in synapse_ids_error_proof_original:
#         final_added_synapses.append(dict(segmentation=2,synapse_id=q["synapse_id"]))

In [ ]:
# print(len(synapse_ids_error_proof_original)),print(len(final_added_synapses))
# print(len(synapse_ids_error_proof_original) + len(final_added_synapses)),print(len(ta3p100.SynapseExclude()))

In [ ]:
# #add the last part of synapses that were not from proofreading to the synapseExclude
# ta3p100.SynapseExcludeFinal.insert(final_added_synapses,skip_duplicates=True)